In [1]:
# Core and data handling
import numpy as np
import pandas as pd
import os
from statistics import stdev

# Visualization
import matplotlib.pyplot as plt
%matplotlib inline

# Machine Learning
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.utils import compute_class_weight
from sklearn.metrics import (
    confusion_matrix, mean_squared_error, precision_recall_curve,
    precision_score, recall_score, accuracy_score, f1_score,
    roc_auc_score, average_precision_score
)

# Imbalanced learning
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SVMSMOTE
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
import lightgbm as lgb

# XGBoost
import xgboost as xgb

# Deep Learning
import torch
import torchvision
import torchvision.utils
import tensorflow as tf

# Dataset
from sklearn.datasets import make_classification

# SVM
from sklearn.svm import SVC
## Functions defined by me 
from model_utils import *
from gan_oversampler import gan_augment_with_existing_resampling

# Check GPU availability
print(f"PyTorch CUDA available: {torch.cuda.is_available()}")

# If you want to drop duplicates in a pandas DataFrame, you would do:
# df = df.drop_duplicates()

C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch CUDA available: True


## 1. Read Data 

In [2]:
df = pd.read_csv('creditcard.csv')

# Take a 10% stratified sample
stratified_sample = df.groupby('Class', group_keys=False).apply(
    lambda x: x.sample(frac=0.2, random_state=11)
)


stratified_sample=stratified_sample.iloc[:,1:]

X=stratified_sample.iloc[:,:-1]
y=stratified_sample.iloc[:,-1:]

y=y['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=11,stratify=y)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_9048\2641008765.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stratified_sample = df.groupby('Class', group_keys=False).apply(


# 1. Modelling with LightGBM algorithm with just using SMOTE to oversample train data 
## For each configuration, model is trained for 20 times and the averages of the output scores are taken.

In [4]:
# Define an SVM with RBF kernel
svm_estimator = SVC(kernel='rbf', gamma='scale', random_state=11)

# Apply SVM-SMOTE with the RBF kernel SVM
svm_smote = SMOTE(
    #svm_estimator=svm_estimator,  # Use the RBF kernel SVM
    random_state=11
)
X_train_resampled, y_train_resampled = svm_smote.fit_resample(X_train, y_train)

In [6]:

sum=0
for i in range(20):
    r=callf1(X_train_resampled,y_train_resampled.ravel(),X_test,y_test.ravel())
    sum+=pd.DataFrame(r.values())
sum=sum/20
r=pd.DataFrame(r.items(),columns=['metric','value'])
r['value']=sum
r

C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\421716341.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r=callf1(X_train_resampled,y_train_resampled.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 39803, number of negative: 39803
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 79606, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (2.43 MB) transferred to GPU in 0.006645 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\421716341.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r=callf1(X_train_resampled,y_train_resampled.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 39803, number of negative: 39803
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 79606, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (2.43 MB) transferred to GPU in 0.006646 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\421716341.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r=callf1(X_train_resampled,y_train_resampled.ravel(),X_test,y_test.ravel())


...

C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\421716341.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r=callf1(X_train_resampled,y_train_resampled.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 39803, number of negative: 39803
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 79606, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (2.43 MB) transferred to GPU in 0.005853 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\421716341.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r=callf1(X_train_resampled,y_train_resampled.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 39803, number of negative: 39803
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 79606, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (2.43 MB) transferred to GPU in 0.005934 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


,metric,value
0,accuracy,0.998830
1,roc_auc,0.998044
2,pr_auc,0.805139
3,recall,0.862069
4,f1,0.714286
5,specificity,0.999062
6,fp_rate,0.000938
7,g_mean,0.928041


In [7]:
basic_smote=r

In [8]:
basic_smote

,metric,value
0,accuracy,0.998830
1,roc_auc,0.998044
2,pr_auc,0.805139
3,recall,0.862069
4,f1,0.714286
5,specificity,0.999062
6,fp_rate,0.000938
7,g_mean,0.928041


# 2. Modelling with LightGBM algorithm with  using SMOTE-GAN hybrids to oversample train data 


In [9]:
from gan_oversampler import gan_augment_with_existing_resampling
from cgan_oversampler_2 import cgan_augment_with_existing_resampling
from wgan_oversampler_3 import wgan_gp_augment


In [10]:
Xn, yn = gan_augment_with_existing_resampling(X_train, y_train, X_train_resampled, y_train_resampled)
Xn_cgan, yn_cgan = cgan_augment_with_existing_resampling(X_train, y_train, X_train_resampled, y_train_resampled)
Xn_wgan, yn_wgan = wgan_gp_augment(X_train, y_train, X_train_resampled, y_train_resampled)

C:\Users\Onur Yaman\Tez\gan_oversampler.py:75: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_tr = y_train.ravel()
100%|██████████| 1/1 [00:00<00:00,  8.68it/s]


Epoch 1, Step 1 | Gen Loss: 1.431168019771576, Disc Loss: 1.370058298110962


100%|██████████| 1/1 [00:00<00:00,  8.58it/s]


Epoch 2, Step 2 | Gen Loss: 0.716127872467041, Disc Loss: 0.6785457134246826


100%|██████████| 1/1 [00:00<00:00,  8.49it/s]


...

Epoch 197 | Gen Loss: -1.7642, Critic Loss: -5.1568


100%|██████████| 1/1 [00:00<00:00, 18.21it/s]


Epoch 198 | Gen Loss: -0.0953, Critic Loss: -5.2270


100%|██████████| 1/1 [00:00<00:00, 18.76it/s]


Epoch 199 | Gen Loss: -1.1300, Critic Loss: -5.4750


In [11]:
sum=0
for i in range(20):
    r2=callf1(Xn,yn.ravel(),X_test,y_test.ravel())
    sum+=pd.DataFrame(r2.values())
sum=sum/20
r2=pd.DataFrame(r2.items(),columns=['metric','value'])
r2['value']=sum

C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\2937115412.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r2=callf1(Xn,yn.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 39803, number of negative: 39803
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 79606, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (2.43 MB) transferred to GPU in 0.005988 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\2937115412.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r2=callf1(Xn,yn.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 39803, number of negative: 39803
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 79606, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (2.43 MB) transferred to GPU in 0.005601 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\2937115412.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r2=callf1(Xn,yn.ravel(),X_test,y_test.ravel())


...

[LightGBM] [Info] Number of positive: 39803, number of negative: 39803
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 79606, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (2.43 MB) transferred to GPU in 0.006008 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\2937115412.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r2=callf1(Xn,yn.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 39803, number of negative: 39803
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 79606, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (2.43 MB) transferred to GPU in 0.005935 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\2937115412.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r2=callf1(Xn,yn.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 39803, number of negative: 39803
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 79606, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (2.43 MB) transferred to GPU in 0.005984 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


In [12]:
sum=0
for i in range(20):
    r3=callf1(Xn_cgan, yn_cgan.ravel(),X_test,y_test.ravel())
    sum+=pd.DataFrame(r3.values())
sum=sum/20
r3=pd.DataFrame(r3.items(),columns=['metric','value'])
r3['value']=sum

C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\500202871.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r3=callf1(Xn_cgan, yn_cgan.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 39803, number of negative: 39803
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 79606, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (2.43 MB) transferred to GPU in 0.005827 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\500202871.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r3=callf1(Xn_cgan, yn_cgan.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 39803, number of negative: 39803
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 79606, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (2.43 MB) transferred to GPU in 0.005818 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\500202871.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r3=callf1(Xn_cgan, yn_cgan.ravel(),X_test,y_test.ravel())


...

[LightGBM] [Info] Number of positive: 39803, number of negative: 39803
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 79606, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (2.43 MB) transferred to GPU in 0.005856 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\500202871.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r3=callf1(Xn_cgan, yn_cgan.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 39803, number of negative: 39803
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 79606, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (2.43 MB) transferred to GPU in 0.005995 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\500202871.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r3=callf1(Xn_cgan, yn_cgan.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 39803, number of negative: 39803
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 79606, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (2.43 MB) transferred to GPU in 0.005976 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


In [13]:
sum=0
for i in range(20):
    r4=callf1(Xn_wgan, yn_wgan .ravel(),X_test,y_test.ravel())
    sum+=pd.DataFrame(r4.values())
sum=sum/20
r4=pd.DataFrame(r4.items(),columns=['metric','value'])
r4['value']=sum


C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\1410152961.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r4=callf1(Xn_wgan, yn_wgan .ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 39803, number of negative: 39803
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 79606, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (2.43 MB) transferred to GPU in 0.005961 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\1410152961.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r4=callf1(Xn_wgan, yn_wgan .ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 39803, number of negative: 39803
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 79606, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (2.43 MB) transferred to GPU in 0.005698 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\1410152961.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r4=callf1(Xn_wgan, yn_wgan .ravel(),X_test,y_test.ravel())


...

[LightGBM] [Info] Number of positive: 39803, number of negative: 39803
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 79606, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (2.43 MB) transferred to GPU in 0.005936 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\1410152961.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r4=callf1(Xn_wgan, yn_wgan .ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 39803, number of negative: 39803
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 79606, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (2.43 MB) transferred to GPU in 0.005939 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\1410152961.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r4=callf1(Xn_wgan, yn_wgan .ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 39803, number of negative: 39803
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 79606, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (2.43 MB) transferred to GPU in 0.005870 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


In [14]:
df_merged=pd.concat([r,r2['value'],r3['value'],r4['value']],axis=1)

In [15]:
df_merged

,metric,value,value,value,value
0,accuracy,0.998830,0.999494,0.999532,0.999239
1,roc_auc,0.998044,0.999461,0.999660,0.999559
2,pr_auc,0.805139,0.916850,0.842263,0.788037
3,recall,0.862069,0.862069,0.862069,0.931034
4,f1,0.714286,0.852572,0.862069,0.805970
5,specificity,0.999062,0.999727,0.999766,0.999355
6,fp_rate,0.000938,0.000273,0.000234,0.000645
7,g_mean,0.928041,0.928350,0.928368,0.964590


In [16]:
df_merged.columns=['Metric','Basic SMOTE','SMOTE-GAN Hybrid','SMOTE-CGAN Hybrid','SMOTE-WGAN_GP Hybrid']

In [17]:
df_merged

,Metric,Basic SMOTE,SMOTE-GAN Hybrid,SMOTE-CGAN Hybrid,SMOTE-WGAN_GP Hybrid
0,accuracy,0.998830,0.999494,0.999532,0.999239
1,roc_auc,0.998044,0.999461,0.999660,0.999559
2,pr_auc,0.805139,0.916850,0.842263,0.788037
3,recall,0.862069,0.862069,0.862069,0.931034
4,f1,0.714286,0.852572,0.862069,0.805970
5,specificity,0.999062,0.999727,0.999766,0.999355
6,fp_rate,0.000938,0.000273,0.000234,0.000645
7,g_mean,0.928041,0.928350,0.928368,0.964590


## 3.  Modelling with LightGBM algorithm with SMOTEENN with adaptive distance weight variants 

In [18]:
from adaptive_smoteenn import smoteenn_with_weighted_voting


In [19]:
distance_algos=['gaussian','inverse','exponential','rank','adaptive']
results=[]
for algo in distance_algos:
    X_resampled, y_resampled = smoteenn_with_weighted_voting(
        X_train, y_train,
        strategy=str(algo),          # 'inverse', 'exponential', 'rank', 'adaptive'
        n_neighbors_smote=5,
        n_neighbors_enn=5,
        alpha=1.0,
        beta=1.0,
        sigma=1.0,                    # for gaussian weighting
        random_state=42
    )
    
    sum=0
    
    for i in range(20):
        
        r=callf1(X_resampled, y_resampled.ravel(),X_test,y_test.ravel())
        sum+=pd.DataFrame(r.values())
    sum=sum/20
    r=pd.DataFrame(r.items(),columns=['metric','value'])
    r['value']=sum

    results.append(r)


Original class distribution: Counter({np.int64(0): 39803, np.int64(1): 69})
After SMOTE: Counter({np.int64(0): 39803, np.int64(1): 39803})
After distance-weighted ENN: Counter({np.int64(1): 39803, np.int64(0): 39746})
[LightGBM] [Info] Number of positive: 39803, number of negative: 39746
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 79549, number of used features: 29


C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\2766025319.py:19: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r=callf1(X_resampled, y_resampled.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (2.43 MB) transferred to GPU in 0.006148 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500358 -> initscore=0.001433
[LightGBM] [Info] Start training from score 0.001433


C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\2766025319.py:19: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r=callf1(X_resampled, y_resampled.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 39803, number of negative: 39746
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 79549, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (2.43 MB) transferred to GPU in 0.005979 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500358 -> initscore=0.001433
[LightGBM] [Info] Start training from score 0.001433


...

[LightGBM] [Info] Number of positive: 39803, number of negative: 39721
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 79524, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (2.43 MB) transferred to GPU in 0.005867 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500516 -> initscore=0.002062
[LightGBM] [Info] Start training from score 0.002062


C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\2766025319.py:19: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r=callf1(X_resampled, y_resampled.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 39803, number of negative: 39721
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 79524, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (2.43 MB) transferred to GPU in 0.006035 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500516 -> initscore=0.002062
[LightGBM] [Info] Start training from score 0.002062


C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\2766025319.py:19: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r=callf1(X_resampled, y_resampled.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 39803, number of negative: 39721
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 79524, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (2.43 MB) transferred to GPU in 0.005915 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500516 -> initscore=0.002062
[LightGBM] [Info] Start training from score 0.002062


In [20]:
import pandas as pd

# Initialize a list to store processed results
processed_results = []

for i, algo in enumerate(distance_algos):
    # Convert the results to a Series with metric names as index
    result_series = results[i].set_index('metric')['value']
    result_series.name = algo  # Name the series after the algorithm
    processed_results.append(result_series)

# Concatenate all series into a DataFrame
final_results = pd.concat(processed_results, axis=1)

# Display the final DataFrame


In [21]:
final_results.columns='SMOTE_ENN_'+final_results.columns

In [22]:
final_results.reset_index(drop=True,inplace=True)

In [ ]:
final_results

In [28]:
df_all=pd.concat([df_merged,final_results],axis=1)

## 4. Modelling with LightGBM with no oversampling technique. Just the base data.

In [24]:
sum=0
for i in range(20):
    
    rson=callf1(X_train , y_train.ravel(),X_test,y_test.ravel())
    sum+=pd.DataFrame(rson.values())
sum=sum/20
rson=pd.DataFrame(rson.items(),columns=['metric','value'])
rson['value']=sum

C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\2108790024.py:4: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  rson=callf1(X_train , y_train.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 69, number of negative: 39803
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 39872, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (1.22 MB) transferred to GPU in 0.004508 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.001731 -> initscore=-6.357591
[LightGBM] [Info] Start training from score -6.357591


C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\2108790024.py:4: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  rson=callf1(X_train , y_train.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 69, number of negative: 39803
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 39872, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (1.22 MB) transferred to GPU in 0.004816 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.001731 -> initscore=-6.357591
[LightGBM] [Info] Start training from score -6.357591


C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\2108790024.py:4: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  rson=callf1(X_train , y_train.ravel(),X_test,y_test.ravel())


...

[LightGBM] [Info] Number of positive: 69, number of negative: 39803
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 39872, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (1.22 MB) transferred to GPU in 0.004484 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.001731 -> initscore=-6.357591
[LightGBM] [Info] Start training from score -6.357591


C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\2108790024.py:4: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  rson=callf1(X_train , y_train.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 69, number of negative: 39803
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 39872, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (1.22 MB) transferred to GPU in 0.004513 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.001731 -> initscore=-6.357591
[LightGBM] [Info] Start training from score -6.357591


C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_21624\2108790024.py:4: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  rson=callf1(X_train , y_train.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 69, number of negative: 39803
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 39872, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (1.22 MB) transferred to GPU in 0.004551 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.001731 -> initscore=-6.357591
[LightGBM] [Info] Start training from score -6.357591


In [24]:
df_all.to_excel('df_all_results.xlsx')

In [25]:
df_all

,Metric,Basic SMOTE,SMOTE-GAN Hybrid,SMOTE-CGAN Hybrid,SMOTE-WGAN_GP Hybrid,SMOTE_ENN_gaussian,SMOTE_ENN_inverse,SMOTE_ENN_exponential,SMOTE_ENN_rank,SMOTE_ENN_adaptive,NO SMOTE
0,accuracy,0.999,0.999,1.000,0.999,0.999,0.999,0.999,0.999,0.999,0.999
1,roc_auc,0.998,0.999,1.000,1.000,0.998,0.999,0.998,0.999,0.999,0.914
2,pr_auc,0.805,0.917,0.842,0.788,0.806,0.813,0.803,0.816,0.813,0.739
3,recall,0.862,0.862,0.862,0.931,0.862,0.862,0.862,0.862,0.862,0.864
4,f1,0.714,0.853,0.862,0.806,0.704,0.729,0.725,0.693,0.727,0.745
5,specificity,0.999,1.000,1.000,0.999,0.999,0.999,0.999,0.999,0.999,0.999
6,fp_rate,0.001,0.000,0.000,0.001,0.001,0.001,0.001,0.001,0.001,0.001
7,g_mean,0.928,0.928,0.928,0.965,0.928,0.928,0.928,0.928,0.928,0.928
